Paired TAM (transcient abnormal myleoproliferation) vs AML (acute myeloid leukemia) analysis of patients with the co-occuring condition of Down Syndrome using DESeq2 on IJC counts obtained from rMATS analysis.

Using a matrix constructed from Kids First Workflow V4 done on single runs, a post-rMATS-single-run prepareSEfiles.sh was run that created a bed file for visualizaiton in UCSC Genome browser of all the events, as well as created a matrix of the single runs normalized to the non-redundant union of files.  Using associative arrays in an awk script, it was a rapid way to transform the individual counts from each of the individual runs into a matrix that facilitated analysis.

Using annotations obtained from the rMATS run that provided the coordinates of each of the splicing events as well as the gene that the junctions came from and the count of the reads that overlapped the junctions.   
 
We will use DESeq2 to perform analysis of these junction counts in the identical way that a gene analysis would be completed.

This allows us to take advantage of the gene-specific shrinkage approach employed by DESeq2 for these junctions.

We will use BiocManager to manage the installation of the Bioconductor package, DeSeq2

In [1]:
if (!require("BiocManager", quietly = TRUE))
    install.packages("BiocManager", repos = "https://cloud.r-project.org")

BiocManager::install("DESeq2")


'getOption("repos")' replaces Bioconductor standard repositories, see
'help("repositories", package = "BiocManager")' for details.
Replacement repositories:
    CRAN: https://cran.r-project.org

Bioconductor version 3.17 (BiocManager 1.30.21.1), R 4.3.1 (2023-06-16)

Warning message:
“package(s) not installed when version(s) same as or greater than current; use
  `force = TRUE` to re-install: 'DESeq2'”


In [2]:
setwd("../data/")


In [3]:
getwd()


[1] "/Users/annedeslattesmays/Desktop/projects/rmats-ijc-analysis/data"

In [4]:
cts <- system.file("SE.IJC.paired.TAM.AML.csv")


In [5]:
anno <- system.file("SE.coordinates.matrix.csv")

In [6]:
cts <- as.matrix(read.csv("SE.IJC.paired.TAM.AML.csv",sep=",",row.names="ID"))

In [7]:
head(cts,2)

,PAUVKY.03A,PAUVKY.40A,PAWSNZ.03A,PAWSNZ.40A,PAUTLA.03A,PAUTLA.40A,PAVUDU.03A,PAVUDU.40A
1,9,15,17,0,44,17,33,19
2,22,19,9,16,26,11,26,17


In [12]:
annot <- read.csv("SE.coordinates.matrix.csv",sep=",",row.names=1)

In [9]:
head (annot,2)


,GeneID,geneSymbol,chr,strand,exonStart_0base,exonEnd,upstreamES,upstreamEE,downstreamES,downstreamEE
,<chr>,<chr>,<chr>,<chr>,<int>,<int>,<int>,<int>,<int>,<int>
1,ENSG00000117620.15,SLC35A3,chr1,+,100007033,100007156,99993536,99993741,100009287,100009323
2,ENSG00000117620.15,SLC35A3,chr1,+,100007033,100007156,99993536,99993741,100011364,100011533


In [10]:
coldata <- read.csv("design_matrix.csv",row.names=1)

In [11]:
coldata


,condition
,<chr>
PAUVKY-03A,TAM
PAUVKY-40A,AML
PAWSNZ-03A,TAM
PAWSNZ-40A,AML
PAUTLA-03A,TAM
PAUTLA-40A,AML
PAVUDU-03A,TAM
PAVUDU-40A,AML


In [ ]:
rownames(coldata)

In [ ]:
rownames(coldata) <-sub("-",".",rownames(coldata))

In [ ]:
colnames(cts)

In [ ]:
all(rownames(coldata) %in% colnames(cts))



In [ ]:
library("DESeq2")
dds <- DESeqDataSetFromMatrix(countData = cts,
                              colData = coldata,
                              design = ~ condition)
dds

In [ ]:
featureData <- data.frame(gene=rownames(cts))

In [ ]:
mcols(dds) <- DataFrame(mcols(dds), featureData)
mcols(dds)


In [ ]:
keep <- rowSums(counts(dds)) >= 10
dds <- dds[keep,]

In [ ]:
head(cts,2)

In [ ]:
dds$condition <- factor(dds$condition, levels = c("TAM","AML"))

In [ ]:
dds <- DESeq(dds)
res <- results(dds, contrast=c("condition","TAM","AML"))
res

In [ ]:
par(mar=c(8,5,2,2))
boxplot(log10(assays(dds)[["cooks"]]), range=0, las=2)

In [ ]:
plotMA(res, ylim=c(-3,3))


In [ ]:
resOrdered <- res[order(res$pvalue),]


In [ ]:
summary(res)


In [ ]:
sum(res$padj < 0.1, na.rm=TRUE)


In [ ]:
write.csv(as.data.frame(resOrdered), 
          file="condition_AML_vs_TAM_results.csv")

In [ ]:
sum(res$padj < 0.05, na.rm=TRUE)

In [ ]:
res05 <- results(dds, alpha=0.05)

In [ ]:
summary(res05)

In [ ]:
resultsNames(dds)

In [ ]:
resLFC <- lfcShrink(dds, coef="condition_AML_vs_TAM", type="apeglm")
resLFC

In [ ]:
plotMA(resLFC, ylim=c(-2,2))


In [ ]:
library("pheatmap")
select <- order(rowMeans(counts(dds,normalized=TRUE)),
                decreasing=TRUE)[1:20]
df <- as.data.frame(colData(dds)[,c("condition")])
pheatmap(assay(ntd)[select,], cluster_rows=FALSE, show_rownames=FALSE,
         cluster_cols=FALSE, annotation_col=df)
